# SETUP

In [1]:
if "google.colab" in str(get_ipython()):
    !pip install pyspark
    !git clone https://github.com/lukebella/YelpRecommenderSystem.git
    !mv YelpRecommenderSystem/* .
    !rm -fr YelpRecommenderSystem


In [2]:
import os

os.environ['KAGGLE_USERNAME'] = "xxxxxx"
os.environ['KAGGLE_KEY'] = "xxxxxx"
!kaggle datasets download -p ./data -d yelp-dataset/yelp-dataset
!unzip -n ./data/yelp-dataset.zip -d ./data


zsh:1: command not found: kaggle
unzip:  cannot find or open ./data/yelp-dataset.zip, ./data/yelp-dataset.zip.zip or ./data/yelp-dataset.zip.ZIP.


In [3]:
review_filename = 'data/yelp_academic_dataset_review.json'
user_filename = 'data/yelp_academic_dataset_user.json'
business_filename = 'data/yelp_academic_dataset_business.json'

# Lavorare su un sample più piccolo (10%)
### conviente farlo solo sulle review per problemi di join

In [4]:
from pyspark.sql import SparkSession

# .master().config('spark.driver.memory', "15g")
spark = SparkSession.builder.getOrCreate()
# sc = pyspark.SparkContext().getOrCreate()
sc = spark.sparkContext


23/12/15 13:57:23 WARN Utils: Your hostname, MacBook-Pro-di-Luca.local resolves to a loopback address: 127.0.0.1; using 192.168.1.128 instead (on interface en0)
23/12/15 13:57:23 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/12/15 13:57:23 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
# TODO add docstring
def from_json_to_RDD(filename):
    raw_df = spark.read.json(filename)
    raw_df.show()
    #return raw_df.sample(frac=1/10, random_state = 0).dropna().rdd
    return raw_df.rdd

raw_review_RDD = from_json_to_RDD(review_filename)#.sample(True, 1, 0)  #699192 rows #shuffle
raw_user_RDD = from_json_to_RDD(user_filename)
raw_business_RDD = from_json_to_RDD(business_filename)
#print(raw_business_RDD.filter(lambda x: x[2]=='bqFG0QJY9jj2m55OqAVHeA').first())

raw_review_RDD.first()

+--------------------+----+-------------------+-----+--------------------+-----+--------------------+------+--------------------+
|         business_id|cool|               date|funny|           review_id|stars|                text|useful|             user_id|
+--------------------+----+-------------------+-----+--------------------+-----+--------------------+------+--------------------+
|XQfwVwDr-v0ZS3_Cb...|   0|2018-07-07 22:09:11|    0|KU_O5udG6zpxOg-Vc...|  3.0|If you decide to ...|     0|mh_-eMZ6K5RLWhZyI...|
|7ATYjTIgM3jUlt4UM...|   1|2012-01-03 15:28:18|    0|BiTunyQ73aT9WBnpR...|  5.0|I've taken a lot ...|     1|OyoGAe7OKpv6SyGZT...|
|YjUWPpI6HXG530lwP...|   0|2014-02-05 20:30:30|    0|saUsX_uimxRlCVr67...|  3.0|Family diner. Had...|     0|8g_iMtfSiwikVnbP2...|
|kxX2SOes4o-D3ZQBk...|   1|2015-01-04 00:01:03|    0|AqPFMleE6RsU23_au...|  5.0|Wow!  Yummy, diff...|     1|_7bHUi9Uuf5__HHc_...|
|e4Vwtrqf-wpJfwesg...|   1|2017-01-14 20:54:15|    0|Sx8TMOWLNuJBWer-0...|  4.0|Cute inter

+-------------+---------------+---------------+----------------+--------------+---------------+---------------+---------------+-----------------+----------------+------------------+-----------------+-----+--------------------+----+--------------------+-----+---------+------------+------+--------------------+-------------------+
|average_stars|compliment_cool|compliment_cute|compliment_funny|compliment_hot|compliment_list|compliment_more|compliment_note|compliment_photos|compliment_plain|compliment_profile|compliment_writer| cool|               elite|fans|             friends|funny|     name|review_count|useful|             user_id|      yelping_since|
+-------------+---------------+---------------+----------------+--------------+---------------+---------------+---------------+-----------------+----------------+------------------+-----------------+-----+--------------------+----+--------------------+-----+---------+------------+------+--------------------+-------------------+
|         

23/12/15 13:57:36 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+--------------------+--------------------+--------------------+--------------------+--------------+--------------------+-------+-------------+--------------+--------------------+-----------+------------+-----+-----+
|             address|          attributes|         business_id|          categories|          city|               hours|is_open|     latitude|     longitude|                name|postal_code|review_count|stars|state|
+--------------------+--------------------+--------------------+--------------------+--------------+--------------------+-------+-------------+--------------+--------------------+-----------+------------+-----+-----+
|1616 Chapala St, ...|{null, null, null...|Pns2l4eNsfO8kk83d...|Doctors, Traditio...| Santa Barbara|                null|      0|   34.4266787|  -119.7111968|Abby Rappoport, L...|      93101|           7|  5.0|   CA|
|87 Grasso Plaza S...|{null, null, null...|mpf3x-BjTdTEA3yCZ...|Shipping Centers,...|        Affton|{8:0-18:30, 0:0-0...|      1|   

Row(business_id='XQfwVwDr-v0ZS3_CbbE5Xw', cool=0, date='2018-07-07 22:09:11', funny=0, review_id='KU_O5udG6zpxOg-VcAEodg', stars=3.0, text="If you decide to eat here, just be aware it is going to take about 2 hours from beginning to end. We have tried it multiple times, because I want to like it! I have been to it's other locations in NJ and never had a bad experience. \n\nThe food is good, but it takes a very long time to come out. The waitstaff is very young, but usually pleasant. We have just had too many experiences where we spent way too long waiting. We usually opt for another diner or restaurant on the weekends, in order to be done quicker.", useful=0, user_id='mh_-eMZ6K5RLWhZyISBhwA')

In [6]:
#sample number of users and take all the reviews made by the users in the subset

# print(raw_user_RDD.count())
raw_user_RDD_bis = raw_user_RDD.filter(lambda x:x[-4]>0).sample(False, 1/500, 0) #withReplacement of .sample True or False?
user_list = raw_user_RDD_bis.map(lambda x: x[-2]).collect()
#3987
raw_review_RDD = raw_review_RDD.filter(lambda x: (x[8] in user_list))
print(len(user_list))
raw_review_RDD.count()
#13608


3987


13608

# RETRIEVE RDDs

In [7]:
import hashlib

def hash(s):
    h = hashlib.sha1(s.encode())
    return int(h.hexdigest(),16)%((2**31)-1)

In [8]:
def get_review_tuple(entry):
    """ Parse a row in the review dataset form pyspark.sql.Row to tuple (remove unintersted columns)
    Args:
        entry (pyspark.sql.types.Row): a row in the review dataset in JSON format
    Returns:
        tuple: (review_id, user_id, business_id, stars, useful, funny, cool, text)
    """

    return (hash(str(entry["review_id"])),    # 0
            hash(str(entry["user_id"])),      # 1
            hash(str(entry["business_id"])),  # 2
            int(entry["stars"]),        # 3
            int(entry["useful"]),       # 4
            int(entry["funny"]),        # 5
            int(entry["cool"]),         # 6
            str(entry["text"]))         # 7


review_RDD = raw_review_RDD.map(get_review_tuple)

review_count = review_RDD.count()

print(f'There are {review_count} reviews in the dataset')
print(f'Reviews: {review_RDD.first()}')


There are 13608 reviews in the dataset
Reviews: (77151601, 444249529, 1016675114, 4, 0, 0, 0, '4-5 stars, this is just after one visit so far, worth a try if you\'re a thai fan. I liked the food, but you always have to have the first visit to see "how they do things" and figure out what your preferences will need to be.\n\nI did use their living social deal so I brought my dad in when he came in from out of town to try it out with me. I like the set up, the quaint house is cute (and well, quaint), but I can see how it wouldn\'t be ideal it if was jam packed. I imagine you could potentially do family style for some of the pub height tabletops. I did call ahead to make sure that I didn\'t need to print out the coupon and it could be used from my ipod and to make sure they were open, as the website said to just double check.\n\nWe started with shrimp spring rolls, probably the best I\'ve had, the ingredients seemed fresher, and crispier. No shredded lettuce here. Simple and loosely wrappe

In [9]:
def get_user_tuple(entry):
    """ Parse a row in the user dataset form pyspark.sql.Row to tuple (remove unintersted columns)
    Args:
        entry (pyspark.sql.types.Row): a row in the user dataset in JSON format
    Returns:
        tuple: (user_id, name, review_count, average_stars, useful, funny, cool, fans)
    """

    return (hash(str(entry["user_id"])),          # 0
            str(entry["name"]),             # 1
            int(entry["review_count"]),     # 2
            float(entry["average_stars"]),  # 3
            int(entry["useful"]),           # 4
            int(entry["funny"]),            # 5
            int(entry["cool"]),             # 6
            int(entry["fans"]))             # 7


user_RDD = raw_user_RDD.map(get_user_tuple)

user_count = user_RDD.count()

print(f'There are {user_count} users in the dataset')
print(f'Users: {user_RDD.first()}')


There are 1987897 users in the dataset
Users: (1328793112, 'Walker', 585, 3.91, 7217, 1259, 5994, 267)


In [10]:
def get_business_tuple(entry):
    """ Parse a row in the business dataset form pyspark.sql.Row to tuple (remove unintersted columns)
    Args:
        entry (pyspark.sql.types.Row): a row in the business dataset in JSON format
    Returns:
        tuple: (business_id, name, city, state, stars, review_count, categories)
    """

    categories = [] if entry["categories"] == None \
                    else str(entry["categories"]).split(", ")
    
    return (hash(str(entry["business_id"])),  # 0
            str(entry["name"]),         # 1
            str(entry["city"]),         # 2
            str(entry["state"]),        # 3
            float(entry["stars"]),      # 4
            int(entry["review_count"]), # 5
            categories)                 # 6

#TODO Attributes?

business_RDD = raw_business_RDD.map(get_business_tuple)

business_count = business_RDD.count()
# print(business_RDD.filter(lambda x: x[0]=='1H8ReY5GlGcHJz7umVidkg').first())

print(f'There are {business_count} business in the dataset')
print(f'Business: {business_RDD.first()}')

There are 150346 business in the dataset
Business: (1768608060, 'Abby Rappoport, LAC, CMQ', 'Santa Barbara', 'CA', 5.0, 7, ['Doctors', 'Traditional Chinese Medicine', 'Naturopathic/Holistic', 'Acupuncture', 'Health & Medical', 'Nutritionists'])


# Calculate overall
### - for each business_id:

$$
  \Delta = \dfrac{1}{2} 
              \left( 
                  \dfrac{\text{useful} + \dfrac{1}{2}(\text{funny} + \text{cool})} 
                        {\text{best\ useful} + \dfrac{1}{2}(\text{best\ funny} + \text{best\ cool})}
                        + 
                  \dfrac{\text{fans}}
                        {\text{best\ fans}}
              \right)
$$
$$
\Delta : [0, 1]
$$
$$
  \text{overall} = \begin{cases} 
              \text{stars} + \Delta & \text{if stars } \ge 3\\ 
              \text{stars} - \Delta & \text{if stars } \lt 3
            \end{cases}
$$

In [11]:
def get_business_id_useful_funny_cool(row):
    """
        Writes a tuple with key business_id and attributes useful, funny, cool
    """
    return (row[2], (row[4], row[5], row[6]))

def get_max_useful_funny_cool(row1, row2):
    """
        Returns a new RDD with, for each business_id key, the maxima values of useful, funny, cool
    """
    return tuple(max(row1[i], row2[i]) for i in range(3))

review_tuple_RDD = review_RDD.map(get_business_id_useful_funny_cool)
review_best_ufc_RDD = review_tuple_RDD.reduceByKey(get_max_useful_funny_cool)

review_best_ufc_RDD.take(5)

# review_best_ufc_RDD tuple: (business_id, (useful, funny, cool)): for each reastaurant, this tuple takes the maxima values of useful, funny and cool. 


[(94502400, (1, 0, 1)),
 (491225560, (0, 0, 0)),
 (1770665880, (0, 0, 0)),
 (1981417000, (6, 2, 2)),
 (299406040, (0, 0, 0))]

In [12]:
def rearrange_review_and_best_ufc_RDD(row):
    """
        Merge, for each business_id, the row values with the best useful, funny, cool ones
    """
    return (row[2], (row[0], row[1], row[3], row[4], row[5], row[6]))

review_ufc_RDD = review_RDD.map(rearrange_review_and_best_ufc_RDD).join(review_best_ufc_RDD)
review_ufc_RDD.first()

# (id_business, ((id_review, id_user, star, useful, funny , cool), (best useful, best funny, best cool)))


(94502400, ((1256229733, 636305136, 5, 0, 0, 0), (1, 0, 1)))

In [13]:
def partial_overall(x):
    """
        Rearrange review_ufc_RDD and calculates the partial overall of the delta above
        partial_overall ={[useful + (funny + cool)/2]/[best useful + (best funny + best cool)/2]
    """
    key = (x[1][0][0], x[1][0][1], x[0], x[1][0][2])
    num = (x[1][0][3] + (x[1][0][4] + x[1][0][5])/2)
    den = (x[1][1][0] + (x[1][1][1] + x[1][1][2])/2)
    return (*key,(num/den if sum(x[1][1]) != 0 else 0))


partial_review_overall_RDD = review_ufc_RDD.map(partial_overall)
partial_review_overall_RDD.first()

# tuple: (review_id, user_id, buisness_id, stars, partial overall) 

(1256229733, 636305136, 94502400, 5, 0.0)

In [14]:
# tuple: (user_id, fans)
user_fans_RDD = user_RDD.map(lambda x: (x[0], x[7]))

# tuple: (user_id, (review_id, buisness_id, stars, partial overall), fans)
partial_overall_fans_RDD = partial_review_overall_RDD.map(lambda x: (x[1], (x[0], x[2], x[3], x[4]))).join(user_fans_RDD)

partial_overall_fans_RDD.first()


(1622563412, ((988434858, 1830274560, 1, 1.0), 0))

In [15]:
# user with the highest number of fans
best_user_fans = user_RDD.max(lambda x: x[7])
print("User with the highest number of fans: ",best_user_fans)


def review_overall(x):
    user_id, review_id, business_id, stars= x[0], x[1][0][0], x[1][0][1], x[1][0][2]
    value = stars + (x[1][0][3]+x[1][1]/best_user_fans[7])/(2 if stars >=3 else -2) 
    return (review_id, user_id, business_id, value)

    #tuple: (review_id, user_id, business_id, stars + (partial overall + fans/best_fans))


review_overall_RDD = partial_overall_fans_RDD.map(review_overall)

review_overall_RDD.sortBy(lambda x:x[3], ascending=False).take(5)


User with the highest number of fans:  (526211061, 'Mike', 1882, 4.39, 22860, 10965, 15350, 12497)


[(1944424370, 1435479599, 1324409120, 5.507921901256301),
 (1795278908, 1298897066, 690862244, 5.507161718812515),
 (812528418, 1298897066, 491347608, 5.507161718812515),
 (490468167, 1298897066, 2087399929, 5.507161718812515),
 (338941442, 1298897066, 1986851853, 5.507161718812515)]

# Basic Recommendation

In [16]:
#Keeping businesses that have at least 10 reviews
threshold = 10
review_by_business_filtered_RDD = review_overall_RDD.map(lambda x: (x[2], (x[0], x[1], x[3]))).groupByKey()\
                                                    .mapValues(list).filter(lambda x: len(x[1])>=threshold)
review_by_business_filtered_RDD.take(5)


[(2102509260,
  [(1003713303, 1046582321, 4.139128946502717),
   (1828544940, 1994483617, 5.167426849110453),
   (1867431700, 1649056566, 5.0),
   (590447591, 41026287, 3.0555555555555554),
   (1915326982, 398441560, 5.502520604945187),
   (586526257, 454274731, 3.1115912263387657),
   (260787533, 953966376, 5.000680163239178),
   (539915488, 374713485, 5.028097854596214),
   (1652614631, 287769425, 5.000080019204609),
   (1854456319, 1476814463, 4.000680163239178),
   (676273809, 975617949, 5.000200048011523)]),
 (279921001,
  [(810169582, 496957576, 5.001240297671441),
   (994433667, 279370217, 4.000080019204609),
   (766004812, 739336230, 2.0),
   (1834201982, 736911376, 5.0),
   (1480686845, 1451210502, 5.000080019204609),
   (942708580, 2130554238, 5.000040009602305),
   (29282503, 1913891406, 3.0),
   (248606800, 158878345, 5.0),
   (29852523, 1167756140, 4.001080259262223),
   (1960528031, 1494199259, 1.999919980795391),
   (299323899, 1133102310, 4.506641593982556),
   (1898443

In [17]:
def sortFunction(tuple):
    """ Construct the sort string (does not perform actual sorting)
    Args:
        tuple: (rating, MovieName)
    Returns:
        sortString: the value to sort with, 'rating MovieName'
    """

    if (tuple[1][1]!= None):
        value = '{:.3f}'.format(tuple[1][1])
    else:
        value = ''
    key = tuple[0]
    return (value + ' ' + str(key))


In [18]:
#Make, for each business, the average of the stars for all the reviews

business_overall_RDD = review_by_business_filtered_RDD.map(lambda x: (x[0],  sum(i[2] for i in x[1])/len(x[1])))
business_overall_RDD.first()

(2102509260, 4.545996493703944)

In [19]:
# top recommended businesses
business_RDD.map(lambda x: (x[0], (x[1], x[6]))).leftOuterJoin(business_overall_RDD).sortBy(sortFunction, False).take(10)  #x[2], x[3],.filter(lambda x: 'Restaurants' in x[1][1])


[(871435685,
  (("Commander's Palace",
    ['French',
     'Restaurants',
     'Cocktail Bars',
     'Nightlife',
     'American (New)',
     'Cajun/Creole',
     'Breakfast & Brunch',
     'American (Traditional)',
     'Bars']),
   4.778273896846353)),
 (98911525, (('Los Agaves', ['Mexican', 'Restaurants']), 4.675262917801128)),
 (1705519317,
  (("Felix's Restaurant & Oyster Bar",
    ['Restaurants', 'Sandwiches', 'Seafood', 'Cajun/Creole']),
   4.6007601824437865)),
 (615074086,
  (('Barbuzzo', ['Mediterranean', 'Restaurants', 'Pizza', 'Italian']),
   4.584707406700685)),
 (2102509260,
  (('Reading Terminal Market',
    ['Candy Stores',
     'Shopping',
     'Department Stores',
     'Fast Food',
     'Beer',
     'Wine & Spirits',
     'Fruits & Veggies',
     'Chinese',
     'Food',
     'Ice Cream & Frozen Yogurt',
     'Desserts',
     'Seafood',
     'Health Markets',
     'Bagels',
     'Cheese Shops',
     'Shopping Centers',
     'Chocolatiers & Shops',
     'Meat Shops',
  

# Collaborative Filtering

In [20]:
review_overall_bis = review_overall_RDD.map(lambda x: (x[1], x[2], x[3]))


In [21]:
#Cosine Distance
def cosine_dist(value):
    """ 
    arg: utente da confrontare
    per ogni utente:
        - user_id, prodotto tra rating di item per stessa chiave di item, lista di rating
        - somma di valori nel numeratore, somma di liste del denominatore
        - prodotto delle somma quadratiche nel denominatore tra i due utenti
        - prendi utenti che hanno qualche voto su un business in comune diverso da zero
    """
    value_review = dict(review_overall_bis.filter(lambda x:x[0] == value).map(lambda x:(x[1],x[2])).collect()) #business: voto
    print(value_review)
    if (len(value_review)==0): 
        return None, None
    user_sim = review_overall_bis.map(lambda x: (x[0], (0 if x[1] not in value_review.keys() \
                                                        else x[2]*value_review[x[1]], [x[2]])))\
                                 .reduceByKey(lambda x,y: (x[0]+y[0], x[1]+y[1]))\
                                 .map(lambda x:(x[0], x[1][0]/((sum(i**2 for i in x[1][1])**0.5) * \
                                                               (sum(i**2 for i in value_review.values())**0.5))))\
                                 .filter(lambda x:x[1]!=0 and x[0]!=value).take(5) #prendiamo i 5 utenti più simili
    # TODO nel repo mettere sviluppi futuri, come prendere tutta la lista di user_sim e confrontare l'RMSE con UV decomprosition
    return sorted(user_sim), value_review

def collaborative_filtering(value):
    #print(value)
    user_sim, value_review = cosine_dist(value)
    if (user_sim == None and value_review == None):
        print("Unable to do the Collab Filtering for this user")
        return
    #print(value_review.keys())
    return review_overall_bis.filter(lambda x: (x[0] in dict(user_sim).keys() and x[1] not in value_review.keys()))\
                             .take(10) #.sortBy(lambda x:x[2],ascending = False)


In [22]:
# print(review_overall_bis.take(10)[5])
business_collab = collaborative_filtering(2104358360) # 79992688
# print(review_overall_RDD.filter(lambda x:x[1]=='Eep1pCr1zMqub16tYgUFgQ').collect())
business_collab
#TODO make print of the business as name, categories...

{68894481: 4.500080019204609, 727016493: 4.500080019204609, 77037223: 1.499919980795391, 804071789: 4.500080019204609, 189868594: 3.500080019204609, 2092104439: 4.000080019204609, 478485880: 4.500080019204609, 408585002: 1.499919980795391, 710205811: 5.500080019204609, 1829050935: 3.500080019204609, 1624278136: 1.499919980795391, 13201739: 4.000080019204609, 321776631: 5.500080019204609, 932159751: 3.000080019204609, 436067754: 3.500080019204609, 671639438: 3.000080019204609, 1990354718: 4.000080019204609}


[(79992688, 506681522, 1.9995999039769545),
 (79992688, 287592569, 5.5004000960230455),
 (79992688, 1693518428, 5.0004000960230455),
 (79992688, 1473121869, 5.5004000960230455),
 (1275193888, 300833041, 4.5),
 (1275193888, 1915938167, 4.5),
 (1275193888, 1191303129, 5.0),
 (1275193888, 2121663450, 4.0),
 (1275193888, 2121663450, 5.0),
 (1275193888, 1461420731, 5.0)]

In [23]:
# recommended_business = sorted(business_collab, key=lambda x : -x[2])
business_RDD.filter(lambda x: x[0] in [i[1] for i in business_collab]).collect()

[(1461420731,
  'Granite City Food & Brewery',
  'Franklin',
  'TN',
  3.5,
  356,
  ['Food',
   'American (New)',
   'Brewpubs',
   'Breakfast & Brunch',
   'Restaurants',
   'Breweries',
   'American (Traditional)']),
 (300833041,
  "Oscar's Sports Grille",
  'Gallatin',
  'TN',
  2.5,
  33,
  ['Bars', 'Nightlife', 'Restaurants', 'Sports Bars']),
 (2121663450,
  'Los Cantaritos',
  'Gallatin',
  'TN',
  4.0,
  55,
  ['Mexican', 'Restaurants']),
 (506681522,
  'Hard Rock Cafe',
  'New Orleans',
  'LA',
  3.0,
  300,
  ['Nightlife',
   'Burgers',
   'Music Venues',
   'American (New)',
   'Arts & Entertainment',
   'Restaurants',
   'Venues & Event Spaces',
   'Food',
   'Cafes',
   'Event Planning & Services',
   'American (Traditional)']),
 (1915938167,
  'Alberto’s',
  'Gallatin',
  'TN',
  4.5,
  67,
  ['French',
   'Patisserie/Cake Shop',
   'Fast Food',
   'Mexican',
   'Food',
   'Brazilian',
   'Modern European',
   'Restaurants']),
 (1693518428,
  "Frenchy's Saltwater Cafe",
 

# UV Decompostion

## Initialization

In [24]:
#We need to train and predict the vectors U and V
# train(trainingRDD, rank, iterations, regularization_parameter)  --> (U,V)
#We need to calculate RMSE between the utility and our matrix

rank = 2
# regularization_parameter = 0.1

U_RDD = user_RDD.map(lambda x: (x[0], [1 for _ in range(rank)]))#.sortBy(lambda x: x[0])
V_RDD = business_RDD.map(lambda x: (x[0], [1 for _ in range(rank)]))#.sortBy(lambda x: x[0])

# U_RDD.take(5)
# V_RDD.take(5)


In [25]:
review_overall_bis = review_overall_RDD.map(lambda x: (x[1], x[2], x[3]))
#review_overall_bis = review_RDD.map(lambda x: (x[1], x[2], x[3]))


In [26]:
#NOTE We are bypassing at the moment the validation set (useful for achieving the best rank)
trainingRDD, testRDD = review_overall_bis.randomSplit([7,3], seed=0)

In [27]:
M_BC = sc.broadcast(trainingRDD.filter(lambda x: x[3]!=0).collect())
#LAST CHECKPOINT!!!


In [28]:
V_BC = sc.broadcast(dict(V_RDD.collect()))


In [29]:
U_BC = sc.broadcast(dict(U_RDD.collect()))


## Performing Optimization

In [30]:
# normalized-->tuple: (user_id, buisness_id, overall - normalized_value)
# review_overall --> tuple: (review_id, user_id, business_id, stars + (partial overall + fans/best_fans))

def update_U(entry):
    id = entry[0]
    U_value = entry[1]
    reviews = [i for i in M_BC.value if i[0] == id]
    for s in range(rank):
        res = 0
        den = 0
        for review in reviews:
            j = review[1] # business_id
            p = 0
            for k in [i for i in range(rank) if i != s]:
                #print(V_BC.value)
                
                p += U_value[k] * V_BC.value[j][k]

            res += V_BC.value[j][s] * (review[2] - p)
            den += V_BC.value[j][s]**2
        U_value[s] = res / (den if den > 0 else 1)\
            #if len(reviews)>0 else 1

    return (id, U_value)


In [31]:
def update_V(entry):
    id = entry[0]
    V_value = entry[1]
    reviews = [i for i in M_BC.value if i[1] == id]

    # out = [1]*rank
    for s in range(rank):
        res = 0
        den = 0
        for review in reviews:
            j = review[0] # user_id
            p = 0
            for k in [i for i in range(rank) if i != s]:
                p += V_value[k] * U_BC.value[j][k]

            res += U_BC.value[j][s] * (review[2] - p)
            den += U_BC.value[j][s]**2
        V_value[s] = (res / (den if den > 0 else 1))\
#            if len(reviews)>0 else 1

    return (id, V_value)


# V_RDD.map(lambda x :  update_V(x)).take(20)

In [32]:
def train(U_RDD,V_RDD,iterations):  
# iterations = 2
    global V_BC, U_BC
    for i in range(iterations):
        V_BC = sc.broadcast(dict(V_RDD.collect()))
        U_RDD = U_RDD.map(update_U)

        U_BC = sc.broadcast(dict(U_RDD.collect()))
        V_RDD = V_RDD.map(update_V)

    return U_RDD.collect(), V_RDD.collect()
    

In [33]:
U,V = train(U_RDD, V_RDD, iterations =2)

In [34]:
U

[(1328793112, [0.0, 0.0]),
 (1730453169, [0.0, 0.0]),
 (421534427, [0.0, 0.0]),
 (1515475869, [0.0, 0.0]),
 (623528084, [0.0, 0.0]),
 (1529476967, [0.0, 0.0]),
 (1950993218, [0.0, 0.0]),
 (543154361, [0.0, 0.0]),
 (428422755, [0.0, 0.0]),
 (502728468, [0.0, 0.0]),
 (2037844578, [0.0, 0.0]),
 (1956851149, [0.0, 0.0]),
 (1804702248, [0.0, 0.0]),
 (877315578, [0.0, 0.0]),
 (640819845, [0.0, 0.0]),
 (1654892307, [0.0, 0.0]),
 (1581395151, [0.0, 0.0]),
 (1212409324, [0.0, 0.0]),
 (945302943, [0.0, 0.0]),
 (1937177532, [0.0, 0.0]),
 (1637631753, [0.0, 0.0]),
 (333169312, [0.0, 0.0]),
 (59372952, [0.0, 0.0]),
 (67330113, [0.0, 0.0]),
 (1345061283, [0.0, 0.0]),
 (1298724169, [0.0, 0.0]),
 (2017722250, [0.0, 0.0]),
 (929099467, [0.0, 0.0]),
 (73769501, [0.0, 0.0]),
 (1429843609, [2.7644669074117263, 0.9868063112357167]),
 (1832823360, [0.0, 0.0]),
 (1182890019, [0.0, 0.0]),
 (1062844705, [0.0, 0.0]),
 (1238388338, [0.0, 0.0]),
 (472611938, [0.0, 0.0]),
 (180056890, [0.0, 0.0]),
 (785048578, [0.

In [35]:
V

[(1768608060, [0.0, 0.0]),
 (674023133, [0.0, 0.0]),
 (1853240994, [0.0, 0.0]),
 (25885286, [0.0, 0.0]),
 (886402536, [0.0, 0.0]),
 (899415875, [0.0, 0.0]),
 (151330391, [0.0, 0.0]),
 (659900618, [0.0, 0.0]),
 (2073408338, [0.0, 0.0]),
 (259476066, [0.0, 0.0]),
 (1270265292, [0.0, 0.0]),
 (901865183, [0.0, 0.0]),
 (399025230, [0.0, 0.0]),
 (1188362622, [0.0, 0.0]),
 (1652935626, [0.0, 0.0]),
 (905182201, [1.1428571428571426, 1.0000000000000004]),
 (464645059, [0.0, 0.0]),
 (1007062082, [0.0, 0.0]),
 (1721829350, [0.0, 0.0]),
 (1979569868, [0.0, 0.0]),
 (464040412, [0.0, 0.0]),
 (904870512, [0.0, 0.0]),
 (464545888, [0.0, 0.0]),
 (358680582, [0.0, 0.0]),
 (395093298, [0.0, 0.0]),
 (2071797272, [0.0, 0.0]),
 (1451936390, [0.0, 0.0]),
 (330641765, [0.0, 0.0]),
 (1630810918, [0.0, 0.0]),
 (1725190102, [0.0, 0.0]),
 (1689700950, [1.4906149486740008, 1.0]),
 (701452773, [0.0, 0.0]),
 (1941251565, [0.0, 0.0]),
 (271351505, [0.6262808712149069, 0.9999999999999998]),
 (2107667707, [0.0, 0.0]),


In [36]:
#Take an user-id and suggest business that he has not reviewed

def predict(u_value, business_value):
    return sum(u_value[i]* business_value[i] for i in range(len(u_value)))


def print_prediction(username, business_name):

    U_dict = dict(U)
    V_dict = dict(V)
    un = user_RDD.filter(lambda x: x[0]== username).first()[1]
    bn = business_RDD.filter(lambda x: x[0]== business_name).first()[2]
    print("User:",un, "----- Business:",bn, ":",predict(U_dict[username], V_dict[business_name]))



In [37]:
#RMSE of UV Decomposition

def rmse(testRDD):
    rmse = 0
    U_dict = dict(U)
    V_dict = dict(V)
    for i in testRDD.collect():
        rmse += (i[2] - predict(U_dict[i[0]], V_dict[i[1]]))**2
    return (rmse/len(M_BC.value))**0.5

print("RMSE:", rmse(testRDD))


RMSE: 2.421804260620688


Try to populate U and V with the average value of review rating

In [38]:
avg_value = review_overall_bis.map(lambda x: (1, x[2])).reduceByKey(lambda x,y:(x+y)/2).first()[1]
avg_value

4.03810722531926

In [39]:
U_RDD = user_RDD.map(lambda x: (x[0], [avg_value for _ in range(rank)]))#.sortBy(lambda x: x[0])
V_RDD = business_RDD.map(lambda x: (x[0], [avg_value for _ in range(rank)]))#.sortBy(lambda x: x[0])


In [40]:
U,V = train(U_RDD, V_RDD, iterations =2)

In [41]:
#RMSE of UV Decomposition
UV_loss = rmse(testRDD)
print("RMSE:", UV_loss)


RMSE: 2.3982532572052504


In [44]:
review_overall_tris = review_overall_bis

In [45]:
trainingRDDALS, validationRDDALS, testRDDALS = review_overall_tris.randomSplit([6, 2, 2], seed=0)

print('Training: %s, validation: %s, test: %s\n' % (trainingRDDALS.count(),
                                                    validationRDDALS.count(),
                                                    testRDDALS.count()))
print(trainingRDDALS.take(3))
print(validationRDDALS.take(3))
print(testRDDALS.take(3))

Training: 8167, validation: 2665, test: 2784

[(1622563412, 1830274560, 0.5), (1622563412, 132774070, 0.5), (79992688, 1693518428, 5.0004000960230455)]
[(79992688, 506681522, 1.9995999039769545), (79992688, 287592569, 5.5004000960230455), (987878448, 131076120, 4.5)]
[(79992688, 1473121869, 5.5004000960230455), (79992688, 932159751, 5.5004000960230455), (25800718, 556134563, 5.0)]


In [46]:
import math

def computeError(predictedRDD, actualRDD):
    """ Compute the root mean squared error between predicted and actual
    Args:
        predictedRDD: predicted ratings for each movie and each user where each entry is in the form
                      (UserID, MovieID, Rating)
        actualRDD: actual ratings where each entry is in the form (UserID, MovieID, Rating)
    Returns:
        RSME (float): computed RSME value
    """
    # Transform predictedRDD into the tuples of the form ((UserID, MovieID), Rating)
    predictedReformattedRDD = predictedRDD.map(lambda i: ((i[0], i[1]), i[2]))

    # Transform actualRDD into the tuples of the form ((UserID, MovieID), Rating)
    actualReformattedRDD = actualRDD.map(lambda i: ((i[0], i[1]), i[2]))

    # Compute the squared error for each matching entry (i.e., the same (User ID, Movie ID) in each
    # RDD) in the reformatted RDDs using RDD transformtions - do not use collect()
    squaredErrorsRDD = (predictedReformattedRDD
                        .join(actualReformattedRDD)
                        .map(lambda i: math.pow(i[1][0] - i[1][1], 2))
                       )

    # Compute the total squared error - do not use collect()
    totalError = squaredErrorsRDD.reduce(lambda a, b: a+b)

    # Count the number of entries for which you computed the total squared error
    numRatings = squaredErrorsRDD.count()

    # Using the total squared error and the number of entries, compute the RSME
    return math.pow(float(totalError) / numRatings, 0.5)

In [47]:
from pyspark.mllib.recommendation import ALS

validationForPredictRDD = validationRDDALS.map(lambda i: (i[0], i[1]))

iterations = 2 # 5
regularizationParameter = 0.1
ranks = [2, 4, 8, 12]
errors = [0, 0, 0, 0]
err = 0
tolerance = 0.02

minError = float('inf')
bestRank = -1
bestIteration = -1
for rank in ranks:
    model = ALS.train(trainingRDDALS, rank, iterations=iterations,
                      lambda_=regularizationParameter)
    predictedRatingsRDD = model.predictAll(validationForPredictRDD)
    error = computeError(predictedRatingsRDD, validationRDDALS)
    errors[err] = error
    err += 1
    print('For rank {} the RMSE is {}'.format(rank, error))
    if error < minError:
        minError = error
        bestRank = rank

print('The best model was trained with rank {}'.format(bestRank))

23/12/15 14:22:54 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
23/12/15 14:22:54 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.VectorBLAS
23/12/15 14:22:54 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK


For rank 2 the RMSE is 4.954520372767565


For rank 4 the RMSE is 4.199034977857051


For rank 8 the RMSE is 4.010206728331361


For rank 12 the RMSE is 3.9030366666328615
The best model was trained with rank 12


In [48]:
myModel = ALS.train(trainingRDDALS, bestRank, iterations=iterations,
                    lambda_=regularizationParameter)
testForPredictingRDDALS = testRDDALS.map(lambda i: (i[0], i[1]))
predictedTestRDDALS = myModel.predictAll(testForPredictingRDDALS)

testRMSE = computeError(testRDDALS, predictedTestRDDALS)

print('The model had a RMSE on the test set of {}'.format(testRMSE))

The model had a RMSE on the test set of 4.012701475907121


## ALS vs UV vs ? collaborative filtering ?

In [50]:
model = ALS.train(trainingRDD, 2, iterations=2,
                      lambda_=regularizationParameter)
testForPredictingRDD = testRDD.map(lambda i: (i[0], i[1]))
predictedRatingsRDD = model.predictAll(testForPredictingRDD)

print("RMSE of ALS: ", computeError(testRDD, predictedRatingsRDD))
print("RMSE of UV: ", UV_loss)

RMSE of ALS:  5.1455274666316
RMSE of UV:  2.3982532572052504
